In [8]:
import pandas as pd

df = pd.read_csv('training_set_VU_DM.csv')



In [9]:
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


In [10]:

df['date_time'] = pd.to_datetime(df['date_time'])
df['time_of_day'] = df['date_time'].dt.hour.apply(get_time_of_day)
df['is_weekend'] = df['date_time'].dt.dayofweek >= 5


df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
df['star_rating_difference'] = df['prop_starrating'] - df['visitor_hist_starrating'].fillna(df['prop_starrating'])
df['is_domestic'] = (df['prop_country_id'] == df['visitor_location_country_id']).astype(int)

print(df[['time_of_day', 'star_rating_difference', 'is_domestic']].head())



  time_of_day  star_rating_difference  is_domestic
0     Morning                     0.0            0
1     Morning                     0.0            0
2     Morning                     0.0            0
3     Morning                     0.0            0
4     Morning                     0.0            0


In [11]:
#Competition features

df[[f'comp{i}_rate' for i in range(1, 9)]].fillna(0, inplace=True)

df['rate_advantage_count'] = (df[[f'comp{i}_rate' for i in range(1, 3)]].fillna(0) > 0).sum(axis=1)
df['rate_disadvantage_count'] = (df[[f'comp{i}_rate' for i in range(1, 3)]].fillna(0) < 0).sum(axis=1)
df['availability_advantage_count'] = (df[[f'comp{i}_inv' for i in range(1, 3)]].fillna(0) > 0).sum(axis=1)
df['avg_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].mean(axis=1, skipna=True)
df['max_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].max(axis=1, skipna=True)
df['min_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].min(axis=1, skipna=True)

df.head

/var/folders/__/6x_r2ddd163c_j6x_mtwstbr0000gn/T/ipykernel_34280/1489280912.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[f'comp{i}_rate' for i in range(1, 9)]].fillna(0, inplace=True)


<bound method NDFrame.head of          srch_id           date_time  site_id  visitor_location_country_id  \
0              1 2013-04-04 08:32:15       12                          187   
1              1 2013-04-04 08:32:15       12                          187   
2              1 2013-04-04 08:32:15       12                          187   
3              1 2013-04-04 08:32:15       12                          187   
4              1 2013-04-04 08:32:15       12                          187   
...          ...                 ...      ...                          ...   
4958342   332785 2013-06-30 19:55:18        5                          219   
4958343   332785 2013-06-30 19:55:18        5                          219   
4958344   332785 2013-06-30 19:55:18        5                          219   
4958345   332785 2013-06-30 19:55:18        5                          219   
4958346   332785 2013-06-30 19:55:18        5                          219   

         visitor_hist_starrating 

In [12]:
df.to_csv('feature_eng_out.csv', index=False)